In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.request import Request, urlopen


def get_mb():
    mb_ncov_news = []
    user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0)'


    req = Request('https://news.mb.com.ph/tag/ncov', headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'})


    content = urlopen(req).read()

    print('reading url')
    soup = BeautifulSoup(content)

    print('reading main news section')
    mydivs = soup.find("main", {'class':"tm-content"})

    print('reading urls in news section')
    mydivs = mydivs.find_all("article", {'class':"uk-article listwiththumb"})
    mb_ncov_news += [url.a['href'] for url in mydivs]

    last_page = int(soup.find("ul", {'class':"uk-pagination"}).text[-3:]) + 1

    for i in range(2, last_page):

        page_link = '/page' + str(i) + '/'

        req = Request('https://news.mb.com.ph/tag/ncov' + page_link, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'})

        content = urlopen(req).read()

        print('reading url')
        soup = BeautifulSoup(content)

        print('reading main news section')
        mydivs = soup.find("main", {'class':"tm-content"})

        print('reading urls in news section')
        mydivs = mydivs.find_all("article", {'class':"uk-article listwiththumb"})
        mb_ncov_news += [url.a['href'] for url in mydivs]

        print(i)
        print(len(mb_ncov_news))
        time.sleep(6)
        
        
    mb_df = pd.DataFrame(columns = ['source_id','date','category','title','author','text'])

    for article in mb_ncov_news:  
        user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) '
        req = Request(article, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'})
        content = urlopen(req).read()
        soup = BeautifulSoup(content)

        try:
            article_id = ''
            date = soup.find('time').text
            category = 'nCov'
            title = soup.find("h1", {'class':"uk-article-title uk-margin-bottom-remove"}).text
            author = soup.find('strong').text
            article = soup.find_all("p")
            text = ' '.join(item.text for item in article)
        except AttributeError:
            continue

        print(title, date)
        mb_df = mb_df.append(pd.Series([article_id,date,category, title,author, text], index = mb_df.columns ), ignore_index=True)
        time.sleep(6)
        
        
    mb_df.to_csv('mb_scraped.csv', index = False)